# Creating a network from a file

In [1]:
import getpass

from roseau.load_flow import *

## From JSON

The first format that can be read is the JSON format. The `from_json` class-method of `ElectricalNetwork` can be used
for that.

In [2]:
en = ElectricalNetwork.from_json("data/LVFeeder00939_summer.json")

Note that you can save an `ElectricalNetwork` instance using the method `to_json` if you want.

An authentification is required. Please contact us at [contact@roseautechnologies.com](mailto:contact@roseautechnologies.com) to get identification.

In [3]:
# Authentication
auth = (input("Please enter your username:"), getpass.getpass("Please enter your password:"))

Then, a load flow can be directly solved.

NB: The server takes time to start so the first call is longer than the others.

In [4]:
en.solve_load_flow(auth=auth)

2

The results are accessible the same way as explained in the [Getting Started](Getting_Started.ipynb) page

In [5]:
en.buses_potentials.head(n=8)

potential
bus_id        phase                           
VoltageSource a       11547.0053084+0.0000000j
              b     -5773.502692-10000.000000j
              c     -5773.502692+10000.000000j
              n       0.000000000+0.000000000j
MVLV03045     a       199.9256500+115.1715310j
              b      -0.222627-00230.72633100j
              c      -199.7033210+115.5561070j
              n       0.000000000+0.000000000j

In [6]:
en.branches_currents.head(n=8)

current1            current2
branch_id      phase                                        
Transformer    a      0.182428-0.375059j -7.624267-1.011447j
               b     -0.415595+0.029482j  2.927451+7.139288j
               c      0.233166+0.345576j  4.724479-6.108946j
               n     N00000000a00000000N -0.027663-0.018894j
LVBranch128683 a      7.624267+1.011447j -7.652458-0.976094j
               b     -2.927451-7.139288j  2.977054+7.137541j
               c     -4.724479+6.108946j  4.693273-6.142546j
               n      0.027663+0.018891j -0.017870-0.018897j

Note that the buses, branches (lines, transformers and switches), and loads can be accessed from the electrical network
using the `buses`, `branches` and `loads` fields.

In [7]:
en.buses

{'VoltageSource': <roseau.load_flow.models.buses.VoltageSource at 0x7fb45415a050>,
 'MVLV03045': <roseau.load_flow.models.buses.Bus at 0x7fb45415beb0>,
 'LVBus004111': <roseau.load_flow.models.buses.Bus at 0x7fb45415b8e0>,
 'LVBus004112': <roseau.load_flow.models.buses.Bus at 0x7fb45415b7f0>,
 'LVBus004113': <roseau.load_flow.models.buses.Bus at 0x7fb411bfed70>,
 'LVBus004114': <roseau.load_flow.models.buses.Bus at 0x7fb411bff130>,
 'LVBus004115': <roseau.load_flow.models.buses.Bus at 0x7fb411bff220>,
 'LVBus004116': <roseau.load_flow.models.buses.Bus at 0x7fb411bff310>}

## Adding elements

It is possible to add new buses, loads, lines, switches, transformers to an existing network using the `add_element`
method of the class `ElectricalNetwork`. For instance, the code below adds a new line and constant power load.

In [8]:
# Connect a new bus to the network
ground = en.special_elements[0]
bus = Bus(id="new_bus", n=4)
load = PowerLoad(id="new_load", n=4, bus=bus, s=[100, 0, 0])
lc = LineCharacteristics.from_name_lv("S_AL_240")
line = ShuntLine(
    id="new_line", n=4, bus1=en.buses["LVBus004111"], bus2=bus, ground=ground, line_characteristics=lc, length=0.5
)
en.add_element(bus)
en.add_element(load)
en.add_element(line)

Then, the load flow can be solved again, and the new results are available.

In [9]:
# Rerun the results
en.solve_load_flow(auth=auth)
en.buses_potentials.head(n=8)

potential
bus_id        phase                           
VoltageSource a       11547.0053084+0.0000000j
              b     -5773.502692-10000.000000j
              c     -5773.502692+10000.000000j
              n       0.000000000+0.000000000j
MVLV03045     a       199.9308510+115.1503680j
              b      -0.222526-00230.72600400j
              c      -199.7079440+115.5551830j
              n       0.000000000+0.000000000j

In [10]:
en.branches_currents.head(n=8)

current1            current2
branch_id      phase                                        
Transformer    a      0.186821-0.370275j -8.068645-1.326657j
               b     -0.420824+0.025835j  2.935977+7.139953j
               c      0.234003+0.344440j  4.660553-6.009908j
               n     N00000000a00000000N  0.472115+0.196612j
LVBranch128683 a      8.068645+1.326657j -8.096830-1.291304j
               b     -2.935977-7.139953j  2.985580+7.138206j
               c     -4.660553+6.009908j  4.629347-6.043510j
               n     -0.472109-0.196615j  0.481900+0.196611j